In [1]:
import time
import json
import requests
import pandas as pd
import plotly.express as px

from enum import Enum
from decimal import Decimal
from dataclasses import dataclass

In [2]:
class Timeframe(Enum):
    HOURLY = "1h"
    DAILY = "1d"
    WEEKLY = "1w"
    MONTHLY = "1M"

@dataclass
class Candle:
    time: int
    open: Decimal
    high: Decimal
    low: Decimal
    close: Decimal
    volume: Decimal

def exluded_markets(check: str, excluded: list[str]) -> bool:
    if len(excluded) > 0:
        for item in excluded:
            if check.startswith(item) or check.endswith(item):
                return True
    return False

class Binance():
    BASEURL = "https://api.binance.com"

    def __init__(self) -> None:
        pass

    def markets(self, market: str, exclude: list[str] = []) -> list[str]:
        url = Binance.BASEURL + "/api/v3/ticker/price"
        r = requests.get(url)

        if r.status_code != 200:
            raise requests.exceptions.HTTPError(r.json())

        data = filter(lambda x: x.endswith(market) and not exluded_markets(x, exclude), map(lambda x: x.get("symbol", ""), r.json()))
        return list(data)

    def kline(self, symbol: str, interval: Timeframe) -> list[Candle]:
        url = Binance.BASEURL + "/api/v3/klines"
        payload = { "symbol": symbol, "interval": interval.value }
        r = requests.get(url, params=payload)

        if r.status_code != 200:
            raise requests.exceptions.HTTPError(r.json())

        klines = []
        for kline in r.json():
            candle = Candle(*kline[:6])
            klines.append(candle)

        return klines

In [3]:
stablecoins = ["TUSD", "BUSD", "USDC", "PAX", "USDP", "DAI", "GUSD", "USDD", "USTC", "UST", "USDS"]

binance = Binance()
markets = binance.markets("USDT", stablecoins)

print(f"Downloading {len(markets)} markets...")
market_data = {}
for i in range(len(markets)):
    if i % 20 == 0:
        time.sleep(5)
    market = markets[i]
    market_data[market] = binance.kline(market, Timeframe.DAILY)
print("Finished downloading.")

Finished downloading.


In [4]:
btcusdt = pd.DataFrame.from_dict(market_data["BTCUSDT"]).set_index("time")

In [5]:
btcusdt

,open,high,low,close,volume
time,,,,,
1639180800000,47140.54000000,49485.71000000,46751.00000000,49389.99000000,28889.19358000
1639267200000,49389.99000000,50777.00000000,48638.00000000,50053.90000000,26017.93421000
1639353600000,50053.90000000,50189.97000000,45672.75000000,46702.75000000,50869.52093000
1639440000000,46702.76000000,48700.41000000,46290.00000000,48343.28000000,39955.98445000
1639526400000,48336.95000000,49500.00000000,46547.00000000,48864.98000000,51629.18100000
...,...,...,...,...,...
1681948800000,28797.10000000,29088.30000000,28010.00000000,28243.65000000,76879.09372000
1682035200000,28243.65000000,28374.02000000,27125.00000000,27262.84000000,77684.76790000
1682121600000,27262.84000000,27882.72000000,27140.35000000,27816.85000000,36023.69686000
